## RusVectōrēs: семантические модели для русского языка

#### Елизавета Кузьменко, Андрей Кутузов

В этом тьюториале мы рассмотрим возможности использования веб-сервиса RusVectōrēs и векторных семантических моделей, которые этот веб-сервис предоставляет пользователям. Наша задача -- от "сырого" текста (т.е. текста без всякой предварительной обработки) прийти к данным, которые мы можем передать векторной модели и получить от неё интересующий нас результат.

Тьюториал состоит из трех частей:
* в первой части мы научимся осуществлять предобработку текстовых файлов так, чтобы в дальнейшем они могли использованы в качестве входных данных для моделей RusVectōrēs.
* во второй части мы научимся работать с векторными моделями и выполнять простые операции над векторами слов, такие как "найти семантические аналоги", "сложить вектора двух слов", "вычислить коэффициент близости между двумя векторами слов". 
* в третьей части мы научимся обращаться к сервису RusVectōrēs через API.

Мы рекомендуем использовать **Python3**, работоспособность тьюториала для **Python2** не гарантируется.

## 1. Предобработка текстовых данных

Функциональность **RusVectōrēs** позволяет пользователям делать запрос к моделям [с одним конкретным словом](https://rusvectores.org/ru/similar/) или [с несколькими словами](https://rusvectores.org/ru/calculator/). С помощью сервиса можно также анализировать отношения между [бóльшим количеством слов](https://rusvectores.org/ru/visual/). Но что делать, если вы хотите обработать очень большую коллекцию текстов или ваша задача не решается при помощи конкретных единичных запросов к серверу, которые можно сделать вручную?

В этом случае можно скачать одну из наших [моделей](https://rusvectores.org/ru/models/), а затем обрабатывать с её помощью тексты локально на вашем компьютере. Однако в этом случае необходимо, чтобы данные, которые подаются на вход модели, были в том же формате, что и данные, на которых эта модель была натренирована.

Вы можете использовать наши готовые скрипты, чтобы из сырого текста получить текст в формате, который можно подавать на вход модели. Скрипты лежат [здесь](https://github.com/akutuzov/webvectors/tree/master/preprocessing). Как следует из их названия, один из скриптов использует для предобработки UDPipe, а другой Mystem. Оба скрипта используют стандартные потоки ввода и вывода, принимают на вход текст, выдают тот же текст, только лемматизированный и с частеречными тэгами. Если же вы хотите детально во всем разобраться и понять, например, в чем разница между UDPipe и Mystem, то читайте далее :)

Предобработка текстов для тренировки моделей осуществлялась следующим образом:
* лемматизация и удаление стоп-слов;
* приведение лемм к нижнему регистру;
* добавление частеречного тэга для каждого слова.

Особого внимания заслуживает последний пункт предобработки. Как можно видеть из таблицы с описанием моделей, частеречные тэги для слов в различных моделях принадлежат к разным тагсетам. Первые модели использовали [тагсет **Mystem**](https://tech.yandex.ru/mystem/doc/grammemes-values-docpage/), затем мы перешли на [**Universal POS tags**](https://universaldependencies.org/u/pos/). В моделях на базе [**fastText**](https://fasttext.cc/) частеречные тэги не используются вовсе.

Давайте попробуем воссоздать процесс предобработки текста на примере рассказа [О. Генри "Русские соболя"](https://rusvectores.org/static/henry_sobolya.txt). Для предобработки мы предлагаем использовать [*UDPipe*](https://ufal.mff.cuni.cz/udpipe), чтобы сразу получить частеречную разметку в виде Universal POS-tags. Сначала установим обертку *UDPipe* для Python:

`pip install ufal.udpipe`

*UDPipe* использует предобученные модели для лемматизации и тэггинга. Вы можете использовать [нашу модель](https://rusvectores.org/static/models/udpipe_syntagrus.model) или обучить свою. 

Чтобы загружать файлы, можно использовать реализацию wget в виде питоновской библиотеки:

`pip install wget`

In [ ]:
import wget

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
text_url = 'https://rusvectores.org/static/henry_sobolya.txt'

modelfile = wget.download(udpipe_url)
textfile = wget.download(text_url)

Перед лемматизацией и тэггингом, наши модели были очищены от пунктуации и возможных ошибок при помощи фильтров [Татьяны Шавриной](https://github.com/TatianaShavrina/taiga). Вы можете увидеть вспомогательные функции для очистки текста в [скрипте для препроцессинга](https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_udpipe.py#L71):

Приступим к собственно предобработке текста. Её можно настроить для своей задачи. Так, например, вы можете не использовать части речи или оставить пунктуацию. Если частеречные тэги вам не нужны, в функции ниже выставьте `keep_pos=False`. Если вам необходимо сохранить знаки пунктуации, выставьте `keep_punct=True`. 

In [3]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False # переменная для запоминания того, что нам встретилось имя собственное
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn


Загружаем модель *UDPipe*, читаем текстовый файл и обрабатываем его. В файле должен содержаться необработанный текст (одно предложение на строку или один абзац на строку).
Этот текст токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход подаётся последовательность разделенных пробелами лемм с частями речи ("зеленый\_NOUN трамвай\_NOUN").

In [4]:
from ufal.udpipe import Model, Pipeline
import os
import re
import sys

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    for line in text:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        print(' '.join(output))

In [9]:
text = open(textfile, 'r', encoding='utf-8').read()
tag_ud(text=text, modelfile=modelfile)


Loading the model...
Processing input...


р_PROPN 
в_ADP
с_ADP
с_ADP
к_ADP
и_CCONJ
быть_NOUN

с_ADP
о_ADP
бы_PART
о_ADP
лист_NOUN
я_PRON


о_ADP


Г_PROPN 
быть_NOUN
называть_NOUN
река_NOUN
и_CCONJ

к_ADP
о_ADP
господин_NOUN
дорога_NOUN
а_CCONJ

с_ADP
и_CCONJ
называть_NOUN
и_CCONJ
быть_NOUN


к_ADP
а_CCONJ
к_ADP

называть_NOUN
о_ADP
число_NOUN
ь_NOUN


господин_NOUN
лист_NOUN
а_CCONJ
с_NOUN
а_CCONJ

метр_NOUN
о_ADP
лист_NOUN
лист_NOUN
и_CCONJ

метр_NOUN
а_CCONJ
к_ADP

к_ADP
и_CCONJ
в_ADP
быть_NOUN
река_NOUN

подобный_NOUN
о_ADP
лист_NOUN
о_ADP
же_PART
и_CCONJ
лист_NOUN
и_CCONJ

метр_NOUN
а_CCONJ
лист_NOUN
Ы_NOUN
шоссе_NOUN
а_CCONJ

бывший_PART
река_NOUN
эра_INTJ
дорога_NOUN
и_CCONJ

называть_NOUN
а_CCONJ

о_ADP
бы_PART
быть_NOUN

лист_NOUN
о_ADP
подобный_NOUN
а_CCONJ
том_NOUN
к_ADP
и_CCONJ


о_ADP
называть_NOUN

в_ADP
Ы_NOUN
называть_NOUN
в_ADP
же_PART
дорога_NOUN
быть_NOUN
называть_NOUN

бы_PART
Ы_NOUN
лист_NOUN

подобный_NOUN
о_ADP
к_ADP
и_CCONJ
называть_NOUN
в_ADP
том_NOUN
ь_NOUN

река_NOUN
я_PRON
дорога_NOUN
Ы_NOUN

бы_PAR

и_CCONJ
метр_NOUN
о_ADP
и_NOUN

господин_NOUN
а_CCONJ
река_NOUN
а_CCONJ
называть_NOUN
том_NOUN
и_CCONJ
быть_NOUN
и_NOUN

и_CCONJ
х_CCONJ

бы_PART
лист_NOUN
а_CCONJ
господин_NOUN
о_ADP
называть_NOUN
а_CCONJ
дорога_NOUN
быть_NOUN
же_PART
называть_NOUN
о_ADP
с_ADP
том_NOUN
и_CCONJ


подобный_NOUN
о_ADP
с_NOUN
в_ADP
о_ADP
лист_NOUN
я_PRON
быть_NOUN
том_NOUN

и_CCONJ
метр_NOUN

том_NOUN
а_CCONJ
к_ADP
же_PART
быть_NOUN


подобный_NOUN
о_ADP
лист_NOUN
ь_NOUN
с_NOUN
в_ADP
я_PRON
с_ADP
ь_NOUN

с_ADP
к_ADP
река_NOUN
о_ADP
метр_NOUN
называть_NOUN
Ы_NOUN
метр_NOUN

лист_NOUN
быть_NOUN
к_ADP
с_ADP
и_CCONJ
к_ADP
о_ADP
называть_NOUN
о_ADP
метр_NOUN

в_ADP

дорога_NOUN
в_ADP
быть_NOUN

с_ADP
о_ADP
том_NOUN
называть_NOUN
и_CCONJ

с_ADP
лист_NOUN
о_ADP
в_ADP


в_ADP
быть_NOUN
с_ADP
том_NOUN
и_CCONJ

метр_NOUN
быть_NOUN
же_PART
дорога_NOUN
в_ADP

с_ADP
о_ADP
бы_PART
о_ADP
и_NOUN

называть_NOUN
быть_NOUN
подобный_NOUN
река_NOUN
и_CCONJ
называть_NOUN
в_ADP
же_PART
дорога_NOUN
быть_NOUN
называть_NOUN
называ

т_PROPN 
река_NOUN
в_ADP
бы_PART
Ы_NOUN

подобный_NOUN
о_ADP
том_NOUN
я_PRON
называть_NOUN
в_ADP
лист_NOUN
о_ADP

дорога_NOUN
Ы_NOUN
метр_NOUN
к_ADP
о_ADP
метр_NOUN



с_NOUN
называть_NOUN
а_CCONJ
число_NOUN
и_CCONJ
том_NOUN


река_NOUN
а_CCONJ
с_NOUN
в_ADP
быть_NOUN
лист_NOUN
и_CCONJ

о_ADP
господин_NOUN
о_ADP
называть_NOUN
ь_NOUN

в_ADP

а_CCONJ
дорога_NOUN
о_ADP
в_ADP
о_ADP
и_NOUN

к_ADP
в_ADP
х_CCONJ
называть_NOUN
быть_NOUN


метр_NOUN
а_CCONJ
лист_NOUN
Ы_NOUN
шоссе_NOUN

бывший_PART
река_NOUN
эра_INTJ
дорога_NOUN
и_CCONJ

о_ADP
бы_PART
быть_NOUN
щ_NOUN
а_CCONJ
лист_NOUN

метр_NOUN
о_ADP
лист_NOUN
лист_NOUN
и_CCONJ

с_ADP
том_NOUN
а_CCONJ
том_NOUN
ь_NOUN

подобный_NOUN
а_CCONJ
и_CCONJ
называть_NOUN
ь_NOUN
к_ADP
о_ADP
и_NOUN


метр_NOUN
а_CCONJ
лист_NOUN
Ы_NOUN
шоссе_NOUN

бы_PART
Ы_NOUN
лист_NOUN

с_ADP
а_CCONJ
метр_NOUN
Ы_NOUN
метр_NOUN

с_ADP
и_CCONJ
лист_NOUN
ь_NOUN
называть_NOUN
Ы_NOUN
метр_NOUN


с_ADP
а_CCONJ
метр_NOUN
Ы_NOUN
метр_NOUN

и_CCONJ
с_NOUN
о_ADP
бы_PART
река_NOUN


быть_NOUN
с_ADP
я_PRON
том_NOUN

подобный_NOUN
я_PRON
том_NOUN
ь_NOUN

дорога_NOUN
о_ADP
лист_NOUN
лист_NOUN
а_CCONJ
река_NOUN
о_ADP
в_ADP


Н_NOUN
а_CCONJ
с_ADP
число_NOUN
быть_NOUN
том_NOUN

о_ADP
дорога_NOUN
быть_NOUN
же_PART
дорога_NOUN
Ы_NOUN

я_PRON

река_NOUN
а_CCONJ
с_NOUN
бы_PART
о_ADP
река_NOUN
число_NOUN
и_CCONJ
в_ADP



в_ADP
с_ADP
быть_NOUN

дорога_NOUN
о_ADP
лист_NOUN
же_PART
называть_NOUN
о_ADP

бы_PART
Ы_NOUN
том_NOUN
ь_NOUN

подобный_NOUN
быть_NOUN
река_NOUN
в_ADP
о_ADP
господин_NOUN
о_ADP

с_ADP
о_ADP
река_NOUN
том_NOUN
а_CCONJ


и_CCONJ
называть_NOUN
а_CCONJ
число_NOUN
быть_NOUN

эра_INTJ
том_NOUN
о_ADP

называть_NOUN
быть_NOUN

дорога_NOUN
лист_NOUN
я_PRON

метр_NOUN
быть_NOUN
называть_NOUN
я_PRON


е_PROPN 
с_ADP
лист_NOUN
и_CCONJ

я_PRON

называть_NOUN
а_CCONJ
число_NOUN
называть_NOUN
в_ADP

река_NOUN
а_CCONJ
бы_PART
о_ADP
том_NOUN
а_CCONJ
том_NOUN
ь_NOUN



том_NOUN
о_ADP
господин_NOUN
дорога_NOUN
а_CCONJ

подобный_NOUN
река_NOUN
о_ADP
щ_NOUN
а_CCONJ
и_NOUN

метр

...

эра_INTJ
том_NOUN
и_CCONJ
метр_NOUN


я_PRON

число_NOUN
быть_NOUN
с_ADP
том_NOUN
называть_NOUN
о_ADP

к_ADP
в_ADP
подобный_NOUN
и_CCONJ
лист_NOUN

эра_INTJ
том_NOUN
о_ADP
том_NOUN

метр_NOUN
быть_NOUN
х_CCONJ


подобный_NOUN
о_ADP
называть_NOUN
я_PRON
том_NOUN
называть_NOUN
о_ADP


Н_NOUN
а_CCONJ
дорога_NOUN
быть_NOUN
называть_NOUN
ь_NOUN

быть_NOUN
господин_NOUN
о_ADP

и_CCONJ

подобный_NOUN
о_ADP
и_NOUN
дорога_NOUN
быть_NOUN
метр_NOUN

подобный_NOUN
река_NOUN
о_ADP
господин_NOUN
в_ADP
лист_NOUN
я_PRON
быть_NOUN
метр_NOUN
с_ADP
я_PRON


метр_NOUN
о_ADP
лист_NOUN
лист_NOUN
и_CCONJ

подобный_NOUN
о_ADP
с_ADP
том_NOUN
а_CCONJ
река_NOUN
а_CCONJ
лист_NOUN
а_CCONJ
с_ADP
ь_NOUN

в_ADP
с_ADP
Ы_NOUN
подобный_NOUN
и_CCONJ
том_NOUN
ь_NOUN

с_ADP
в_ADP
о_ADP
и_CCONJ

подобный_NOUN
о_ADP
дорога_NOUN
о_ADP
с_NOUN
река_NOUN
быть_NOUN
называть_NOUN
и_CCONJ
я_PRON


с_ADP
о_ADP
бы_PART
о_ADP
лист_NOUN
я_PRON

х_CCONJ
о_ADP
река_NOUN
о_ADP
шоссе_NOUN
о_ADP

в_ADP
бы_PART
а_CCONJ
ю_VERB
к_ADP
и_CCONJ
в

том_NOUN
называть_NOUN
о_ADP
с_ADP
и_CCONJ
лист_NOUN
и_CCONJ
с_ADP
ь_NOUN

к_ADP

р_PROPN 
эра_INTJ
называть_NOUN
с_ADP
о_ADP
метр_NOUN
в_ADP

с_ADP

с_ADP
и_CCONJ
метр_NOUN
подобный_NOUN
а_CCONJ
том_NOUN
и_CCONJ
быть_NOUN
и_NOUN

и_CCONJ


с_ADP
лист_NOUN
в_ADP
число_NOUN
а_CCONJ
лист_NOUN
о_ADP
с_ADP
ь_NOUN


подобный_NOUN
о_ADP
дорога_NOUN
с_ADP
к_ADP
а_CCONJ
с_NOUN
Ы_NOUN
в_ADP
а_CCONJ
лист_NOUN
и_CCONJ

быть_NOUN
метр_NOUN
в_ADP


к_ADP
в_ADP
дорога_NOUN
а_CCONJ

о_ADP
называть_NOUN

дорога_NOUN
о_ADP
лист_NOUN
же_PART
быть_NOUN
называть_NOUN

называть_NOUN
а_CCONJ
подобный_NOUN
река_NOUN
а_CCONJ
в_ADP
и_CCONJ
том_NOUN
ь_NOUN

с_ADP
в_ADP
о_ADP
и_CCONJ

с_ADP
том_NOUN
о_ADP
подобный_NOUN
Ы_NOUN




Ч_NOUN
том_NOUN
о_ADP

эра_INTJ
том_NOUN
о_ADP

с_NOUN
а_CCONJ

в_ADP
о_ADP
лист_NOUN
называть_NOUN
быть_NOUN
называть_NOUN
и_CCONJ
быть_NOUN

том_NOUN
а_CCONJ
метр_NOUN

называть_NOUN
а_CCONJ

в_ADP
господин_NOUN
лист_NOUN
в_ADP




с_ADP
подобный_NOUN
река_NOUN
о_ADP
с_ADP
и_CCONJ
лис

называть_NOUN
о_ADP

с_ADP

том_NOUN
о_ADP
бы_PART
о_ADP
и_NOUN


по_NOUN
о_ADP

о_ADP
подобный_NOUN
и_CCONJ
с_ADP
а_CCONJ
называть_NOUN
и_CCONJ
ю_VERB

о_ADP
называть_NOUN

о_ADP
число_NOUN
быть_NOUN
называть_NOUN
ь_NOUN

подобный_NOUN
о_ADP
х_CCONJ
о_ADP
же_PART

называть_NOUN
а_CCONJ

эра_INTJ
том_NOUN
и_CCONJ

метр_NOUN
быть_NOUN
х_CCONJ
а_CCONJ


к_ADP
о_ADP
том_NOUN
о_ADP
река_NOUN
Ы_NOUN
быть_NOUN

в_ADP
к_ADP
река_NOUN
а_CCONJ
шоссе_NOUN
а_CCONJ
ю_VERB
том_NOUN

том_NOUN
в_ADP
о_ADP
ю_VERB

дорога_NOUN
быть_NOUN
в_ADP
в_ADP
шоссе_NOUN
к_ADP
в_ADP




по_NOUN
о_ADP
дорога_NOUN
и_CCONJ

том_NOUN
Ы_NOUN


подобный_NOUN
о_ADP
дорога_NOUN
и_CCONJ

том_NOUN
Ы_NOUN

к_ADP

число_NOUN
быть_NOUN
река_NOUN
том_NOUN
в_ADP




с_NOUN
а_CCONJ
подобный_NOUN
а_CCONJ
лист_NOUN
ь_NOUN
число_NOUN
и_CCONJ
в_ADP
о_ADP

с_ADP
к_ADP
а_CCONJ
с_NOUN
а_CCONJ
лист_NOUN

метр_NOUN
а_CCONJ
лист_NOUN
Ы_NOUN
шоссе_NOUN




т_PROPN 
Ы_NOUN

с_NOUN
называть_NOUN
а_CCONJ
быть_NOUN
шоссе_NOUN
ь_NOUN


р_PROPN 


лист_NOUN
Ы_NOUN
шоссе_NOUN
а_CCONJ
лист_NOUN


число_NOUN
том_NOUN
о_ADP

подобный_NOUN
река_NOUN
о_ADP
подобный_NOUN
а_CCONJ
лист_NOUN
и_CCONJ

с_ADP
о_ADP
бы_PART
о_ADP
лист_NOUN
я_PRON


т_PROPN 
а_CCONJ
к_ADP

том_NOUN
Ы_NOUN

и_CCONJ
х_CCONJ

называть_NOUN
а_CCONJ
шоссе_NOUN
быть_NOUN
лист_NOUN


к_ADP
о_ADP
в_ADP
называть_NOUN

подобный_NOUN
река_NOUN
и_CCONJ
подобный_NOUN
о_ADP
дорога_NOUN
называть_NOUN
я_PRON
лист_NOUN

называть_NOUN
а_CCONJ

лист_NOUN
а_CCONJ
дорога_NOUN
о_ADP
называть_NOUN
и_CCONJ

к_ADP
о_ADP
называть_NOUN
быть_NOUN
центнер_NOUN

с_ADP
о_ADP
бы_PART
о_ADP
лист_NOUN
ь_NOUN
быть_NOUN
господин_NOUN
о_ADP

бы_PART
о_ADP
а_CCONJ



бы_PART
Ы_NOUN
в_ADP
шоссе_NOUN
быть_NOUN
и_NOUN

с_ADP
о_ADP
бы_PART
с_ADP
том_NOUN
в_ADP
быть_NOUN
называть_NOUN
называть_NOUN
о_ADP
с_ADP
том_NOUN
и_CCONJ

метр_NOUN
о_ADP
лист_NOUN
лист_NOUN
и_CCONJ

метр_NOUN
а_CCONJ
к_ADP


к_ADP
и_CCONJ
в_ADP
быть_NOUN
река_NOUN



и_CCONJ

в_ADP
называть_NOUN
и_CCONJ
метр_NOUN
а_CCONJ
том_NOUN


с_ADP
и_CCONJ
я_PRON
ю_VERB
щ_NOUN
быть_NOUN
господин_NOUN
о_ADP

в_ADP
с_NOUN
о_ADP
река_NOUN
а_CCONJ

с_ADP

метр_NOUN
а_CCONJ
лист_NOUN
Ы_NOUN
шоссе_NOUN
а_CCONJ


о_ADP
называть_NOUN
а_CCONJ

господин_NOUN
река_NOUN
а_CCONJ
центнер_NOUN
и_CCONJ
о_ADP
с_NOUN
называть_NOUN
Ы_NOUN
метр_NOUN

же_PART
быть_NOUN
с_ADP
том_NOUN
о_ADP
метр_NOUN


дорога_NOUN
о_ADP
с_ADP
том_NOUN
о_ADP
и_NOUN
называть_NOUN
Ы_NOUN
метр_NOUN

господин_NOUN
быть_NOUN
река_NOUN
центнер_NOUN
о_ADP
господин_NOUN
и_CCONJ
называть_NOUN
и_CCONJ


называть_NOUN
а_CCONJ
бы_PART
река_NOUN
о_ADP
с_ADP
и_CCONJ
лист_NOUN
а_CCONJ

называть_NOUN
а_CCONJ

подобный_NOUN
лист_NOUN
быть_NOUN
число_NOUN
и_CCONJ

бы_PART
о_ADP
а_CCONJ


подобный_NOUN
быть_NOUN
река_NOUN
быть_NOUN
к_ADP
и_CCONJ
называть_NOUN
в_ADP
в_ADP

о_ADP
дорога_NOUN
и_CCONJ
называть_NOUN

к_ADP
о_ADP
называть_NOUN
быть_NOUN
центнер_NOUN

с_NOUN
а_CCONJ

с_ADP
подобный_NOUN
и_CCONJ
называть_NOUN
в_ADP




по_NOUN
а_CCONJ
река_NOUN
а_CCONJ

метр_NOUN
о_ADP
ли

UDPipe позволяет нам распознавать имена собственные, и несколько идущих подряд имен мы можем склеить в одно.
Вместо UDPipe возможно использовать и Mystem (удобнее использовать [pymystem](https://pypi.python.org/pypi/pymystem3) для Python), хотя Mystem имена собственные не распознает. Для того чтобы работать с последними моделями RusVectōrēs, понадобится сконвертировать тэги Mystem в UPOS. Кроме того, в данный момент мы не используем Mystem в своей работе, поэтому его совместимость с новыми моделями не гарантируется.

Сначала нужно установить библиотеку pymystem:

`pip install pymystem3`

Затем импортируем эту библиотеку и анализируем с её помощью текст:

In [10]:
from pymystem3 import Mystem

def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            tagged.append(lemma.lower() + '_' + pos)
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

In [11]:
processed_mystem = tag_mystem(text=text)
print(processed_mystem[:10])

['русский_S', 'соболь_S', 'о_PR', 'генри_S', 'когда_CONJ', 'синий_A', 'как_CONJ', 'ночь_S', 'глаз_S', 'молль_S']


Как видно, тэги Mystem отличаются от Universal POS-tags, поэтому следующим шагом должна быть их конвертация в Universal Tags. Вы можете воспользоваться вот [этой таблицей конвертации](https://github.com/akutuzov/universal-pos-tags/blob/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map):

In [12]:
import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

print(mapping)

{'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}


Теперь усовершенствуем нашу функцию `tag_mystem`:

In [13]:
def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

In [14]:
processed_mystem = tag_mystem(text=text)
print(processed_mystem[:10])

['русский_NOUN', 'соболь_NOUN', 'о_ADP', 'генри_NOUN', 'когда_SCONJ', 'синий_ADJ', 'как_SCONJ', 'ночь_NOUN', 'глаз_NOUN', 'молль_NOUN']


Теперь частеречные тэги в тексте, проанализированном при помощи Mystem, сравнимы с тэгами Unversal POS (хотя сам анализ оказался разным)!

При необходимости вы можете также произвести для Mystem точно такую же предобработку текста, которая выше была описана для UDPipe. Вы также можете удалить стоп-слова, воспользовавшись, например, [списком стоп-слов в библиотеке NLTK](https://pythonspot.com/nltk-stop-words/) или на основании того, что слово было распознано как функциональная часть речи (именно так производилась фильтрация в новых моделях).

Итак, в ходе этой части тьюториала мы научились от "сырого текста" приходить к лемматизированному тексту с частеречными тэгами, который уже можно подавать на вход модели! Теперь приступим непосредственно к работе с векторными моделями.

## 2. Работа с векторными моделями при помощи библиотеки Gensim

Прежде чем переходить к работе непосредственно с **RusVectōrēs**, мы посмотрим на то, как работать с дистрибутивными моделями при помощи существующих библиотек. 

Для работы с эмбеддингами слов существуют различные библиотеки: [gensim](https://radimrehurek.com/gensim/), [keras](https://keras.io/), [tensorflow](https://www.tensorflow.org/), [pytorch](https://pytorch.org/). Мы будем работать с библиотекой *gensim*, ведь в основе нашего сервера именно она и используется.


***Gensim***  - изначально библиотека для тематического моделирования текстов. Однако помимо различных алгоритмов для *topic modeling* в ней реализованы на python и алгоритмы из тулкита *word2vec* (который в оригинале был написан на C++). Прежде всего, если *gensim* у вас на компьютере не установлен, нужно его установить:

`pip install gensim`

Gensim регулярно обновляется, так что не будет лишним удостовериться, что у вас установлена последняя версия, а при необходимости проапдейтить библиотеку:

`pip install gensim --upgrade` 

или 

`pip install gensim -U`

При подготовке этого тьюториала использовался *gensim* версии 3.7.0.

Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий. Для этого используется стандартная питоновская библиотека `logging`.

In [18]:
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Работа с моделью

Для своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами. Но для каких-то общих целей модели уже есть как для русского языка, так и для английского.

Модели для русского скачать можно здесь - https://rusvectores.org/ru/models/

Существуют несколько форматов, в которых могут храниться модели. Во-первых, данные могут храниться в нативном формате *word2vec*, при этом модель может быть бинарной или не бинарной. Для загрузки модели в формате *word2vec* в классе `KeyedVectors` (в котором хранится большинство относящихся к дистрибутивным моделям функций) существует функция `load_word2vec_format`, а бинарность модели можно указать в аргументе `binary` (внизу будет пример). Помимо этого, модель можно хранить и в собственном формате *gensim*, для этого существует класс `Word2Vec` с функцией `load`. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте. Наш код определяет тип модели по её расширению, но вообще файл с моделью может называться как угодно, жестких ограничений для расширения нет.

Для новых моделей мы перешли на загрузку с использованием инфраструктуры Nordic Language Processing Laboratory. На практике это означает, что теперь по клику на модель вы скачиваете zip-архив с уникальным числовым идентификатором (например, `180.zip`). Внутри архива всегда находится файл `meta.json`, содержащий в структурированном и стандартном виде информацию о модели и корпусе, на котором она обучена. word2vec-модели лежат в архивах сразу в двух word2vec-форматах: бинарном `model.bin` (удобен для быстрой загрузки) и текстовом `model.txt` (удобен для просмотра человеком). Давайте скачаем новейшую модель для русского языка, созданную на основе [Национального корпуса русского языка (НКРЯ)](http://www.ruscorpora.ru/), и загрузим в её в память. Распаковывать скачанный архив для обычных моделей не нужно, так как его содержимое прочитается при помощи специальной инструкции:

In [19]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

2020-09-27 10:14:33,710 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2020-09-27 10:14:46,210 : INFO : loaded (189193, 300) matrix from <zipfile.ZipExtFile [closed]>


Модели `fasttext` в новой версии gensim загружаются при помощи следующей команды::

`gensim.models.KeyedVectors.load("model.model")`

Перед загрузкой скачанный архив с моделью `fasttext` **необходимо распаковать**. Определить необходимый для загрузки файл несложно, чаще всего это файл с расширением `.model` (остальные файлы из архива должны быть в той же папке). 

Вернемся к нашей модели, созданной на основе НКРЯ. Скажем, нам интересны такие слова (пример для русского языка):

In [22]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студенческий_ADJ']

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [23]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

день_NOUN
неделя_NOUN 0.7375996112823486
день_PROPN 0.7067667245864868
месяц_NOUN 0.7037326693534851
час_NOUN 0.6643950343132019
утро_NOUN 0.6526744961738586
вечер_NOUN 0.6038411855697632
сутки_NOUN 0.5923081040382385
воскресенье_NOUN 0.5842781066894531
полдень_NOUN 0.5743687152862549
суббота_NOUN 0.534594714641571


ночь_NOUN
ночь_PROPN 0.8310785889625549
вечер_NOUN 0.7183678150177002
рассвет_NOUN 0.696594774723053
ночи_NOUN 0.6920218467712402
полночь_NOUN 0.6704976558685303
ночь_VERB 0.6615264415740967
утро_NOUN 0.6263935565948486
ночной_ADJ 0.6024709343910217
полдень_NOUN 0.5835085511207581
сумерки_NOUN 0.5671443343162537


человек_NOUN
человек_PROPN 0.7850059270858765
человеческий_ADJ 0.5915265679359436
существо_NOUN 0.573693037033081
народ_NOUN 0.5354466438293457
личность_NOUN 0.5296981334686279
человечество_NOUN 0.5282931327819824
человкъ_PROPN 0.5047001838684082
индивидуум_NOUN 0.5000404715538025
нравственный_ADJ 0.4972919821739197
потому_ADV 0.49293622374534607


семантика_NOUN

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [24]:
model.most_similar(positive=['студенческий_ADJ'], topn=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('университетский_ADJ', 0.6642225384712219),
 ('студент_NOUN', 0.6486333012580872),
 ('студенчество_NOUN', 0.6344770789146423),
 ('институтский_ADJ', 0.6142880320549011),
 ('гимназический_ADJ', 0.5510081648826599),
 ('аспирантский_ADJ', 0.5403808951377869),
 ('школьный_ADJ', 0.5198260545730591),
 ('студентский_ADJ', 0.5004373788833618),
 ('ифли_PROPN', 0.48894092440605164),
 ('молодежный_ADJ', 0.47924578189849854)]

Находим косинусную близость пары слов:

In [25]:
print(model.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.22025344


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Найди лишнее!

In [26]:
print(model.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN картофель_NOUN'.split()))

картофель_NOUN


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Реши пропорцию!

In [27]:
print(model.most_similar(positive=['пицца_NOUN', 'россия_NOUN'], negative=['италия_NOUN'])[0][0])

гамбургер_NOUN


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## 3. Использование API сервиса RusVectōrēs

Помимо локального использования модели, вы можете также обратиться к RusVectōrēs через API, чтобы использовать наши модели в автоматическом режиме, не скачивая их (скажем, из ваших скриптов). В нашем API имеется две функции:

* получение списка семантически близких слов для заданного слова в заданной модели;
* вычисление коэффициента косинусной близости между парой слов в заданной модели.

Для того чтобы получить список семантически близких слов, необходимо выполнить GET-запрос по адресу в следующем формате:

`https://rusvectores.org/MODEL/WORD/api/FORMAT/`

Разберемся с компонентами этого запроса. `MODEL` - идентификатор модели, к которой мы хотим обратиться. Идентификаторы можно посмотреть в [таблице](https://rusvectores.org/ru/models/) со всеми моделями нашего сервиса. `WORD` - слово, для которого мы хотим узнать соседей. Следует помнить, что частеречный тэг здесь тоже нужен (точнее, вы можете отправлять запросы и без него, но тогда части речи ваших слов сервер определит автоматически - и не всегда правильно). `FORMAT` - формат выходных данных, в настоящий момент это *csv* (с разделением через табуляцию) либо *json*.

Попробуем узнать семантических соседей для первых слов из нашего рассказа. Сначала создадим переменные с параметрами нашего запроса.

In [ ]:
print(processed_ud[:15])
MODEL = 'ruscorpora_upos_cbow_300_20_2019'
FORMAT = 'csv'
WORD = processed_ud[1]

In [ ]:
def api_neighbor(m, w, f):
    neighbors = {}
    url = '/'.join(['http://rusvectores.org', m, w, 'api', f]) + '/'
    r = requests.get(url=url, stream=True)
    for line in r.text.split('\n'):
        try: # первые две строки в файле -- служебные, их мы пропустим
            word, sim = re.split('\s+', line) # разбиваем строку по одному или более пробелам
            neighbors[word] = sim
        except:
            continue
    return neighbors

In [ ]:
print(api_neighbor(MODEL, WORD, FORMAT))

API по умолчанию сообщает 10 ближайших соседей, изменить это количество в данный момент возможности нет.

Теперь рассмотрим вторую функцию, доступную в API - вычисление коэффициента близости между двумя словами.
Запросы для неё должны выполняться в таком виде:

`https://rusvectores.org/MODEL/WORD1__WORD2/api/similarity/`

Здесь переменные - `MODEL` (идентификатор модели, к которой мы обращаемся) и два слова (вместе с их частеречными тэгами). Обратите внимание, что слова разделены **двумя нижними подчеркиваниями**.

In [ ]:
def api_similarity(m, w1, w2):
    url = '/'.join(['https://rusvectores.org', m, w1 + '__' + w2, 'api', 'similarity/'])
    r = requests.get(url, stream=True)
    return r.text.split('\t')[0]

In [ ]:
MODEL = 'tayga_upos_skipgram_300_2_2019'
api_similarity(MODEL, WORD, 'мех_NOUN')

В этом тьюториале мы научились обрабатывать тексты таким образом, чтобы их можно было отдавать в качестве входных данных моделям RusVectōrēs. Мы также рассмотрели основные операции над векторами слов в дистрибутивных семантических моделях и научились обращаться к сервису через API. Надеемся, что данный тьюториал подготовил вас к работе над вашими данными и к новым открытиям, которые можно совершить при помощи дистрибутивной семантики :)